## 1. Absolute vs. relative paths

An **absolute path** starts at the filesystem root (`/` on Unix, `C:\` on Windows) and uniquely identifies a file no matter the current working directory.  A **relative path** is interpreted **relative to `os.getcwd()`** (the process’s current folder).  Mixing them is a common source of *“file not found”* errors when scripts run from cron, IDEs, or CI—always build absolute paths for anything outside working dir.

```python
import os, pathlib
print('cwd =', os.getcwd())
rel = pathlib.Path('README.md')
abs_path = rel.resolve()
print('Absolute:', abs_path)
```

### Quick check

1. Changing working directory with `os.chdir()` affects **relative** path resolution?

2. True / False An absolute path can contain `..` segments.

<details><summary>Answer key</summary>

1. **Yes**—relative paths are evaluated after the cwd change.
2. **True**—though `Path.resolve()` will normalise them away.

</details>

## 2. Opening and closing files with `open()` and `with`

Always wrap `open()` in a `with` block: it guarantees the file descriptor is **closed** even if an exception happens.  Leaving files open leaks OS resources and can corrupt data on Windows where open handles lock the file.

```python
with open('notes.txt', 'w') as f:
    f.write('remember to close me')
# file is closed here
```

### Quick check

1. `with open(...) as f` closes the file:
  a. at end of block   b. when GC runs

2. True / False Calling `f.close()` manually *inside* the block is necessary.

<details><summary>Answer key</summary>

1. **a**.
2. **False**—context manager handles it.

</details>

## 3. Read vs. write vs. append modes

`open(path, 'r')` reads existing file; `'w'` **truncates** or creates; `'a'` appends to end.  Add `'+'` for read+write. Forgetting and opening logs in `'w'` is a classic *data wipe* bug.

```python
# demo: append preserves previous content
with open('demo.log', 'w') as f:
    f.write('first\n')
with open('demo.log', 'a') as f:
    f.write('second\n')
print(open('demo.log').read())
```

### Quick check

1. Mode `'w'` on non‑existent file will:
  a. raise error   b. create file

2. True / False `'r+'` truncates the file on open.

<details><summary>Answer key</summary>

1. **b**.
2. **False**—it preserves size.

</details>

## 4. Text vs. binary mode & encodings

`'t'` (default) decodes bytes to `str` using a chosen **encoding** (UTF‑8 by default on modern Python).  `'b'` returns raw `bytes`.  Use binary when reading images, PDFs, or when you intend to handle encoding yourself.

```python
data = bytes([0x48,0x49])
with open('bin.dat','wb') as f: f.write(data)
print(open('bin.dat','rb').read(), open('bin.dat','rt', encoding='latin-1').read())
```

### Quick check

1. Opening image.jpg in text mode likely raises:
  a. UnicodeDecodeError  b. FileNotFoundError

2. True / False `open(..., 'rb')` returns `str` objects.

<details><summary>Answer key</summary>

1. **a**.
2. **False**—returns `bytes`.

</details>

## 5. Line-by-line iteration vs. `read()` vs. `readlines()`

Iterating `for line in f:` streams one line at a time—constant memory.  `f.read()` loads the **entire file** into memory; dangerous for GB logs.  `f.readlines()` returns list of lines; also memory‑heavy.

```python
big = open(__file__)  # this script
first3 = [next(big) for _ in range(3)]
print(first3)
```

### Quick check

1. Which method is **most** memory efficient for huge files?
  a. read()   b. readlines()   c. iteration

2. True / False `readlines()` strips newline characters.

<details><summary>Answer key</summary>

1. **c**.
2. **False**—lines keep trailing `\n`.

</details>

## 6. Common text encodings & the `encoding=` param

UTF‑8 dominates, but CSVs from Excel may be UTF‑16‑LE or old Latin‑1.  Always pass `encoding=` when data source is known; relying on locale leads to bugs on other machines.

```python
# Write Polish word in UTF‑8
word = 'żółć'
open('pl.txt','w', encoding='utf-8').write(word)
print(open('pl.txt','rb').read())
```

### Quick check

1. UTF‑8 encodes ASCII bytes **unchanged**?
  a. True   b. False

2. True / False Passing wrong encoding causes silent data corruption but no exception.

<details><summary>Answer key</summary>

1. **a**.
2. **True**—bytes decode to mojibake.

</details>